In [7]:
import camb
import numpy as np
%pylab inline
import matplotlib.pyplot as plt
import sys, os, platform, time
from camb import model
from Tinker_MF import dn_dlogM
from Tinker_MF import tinker_params

class CosmoParams:
    def __init__(self,opt):
        h100 = 100. 
        self.rho_crit0 = 3. / (8 * np.pi) * (h100 * 1e5)**2 / c.G_CGS * c.MPC2CM / c.MSUN_CGS
        if (opt == 'WMAP9'):
            self.om = 0.279
            self.ol = 0.721
            self.h0 = 70.0
            self.s8 = 0.821
        if (opt == 'Planck15'):
            self.om = 0.315
            self.ol = 0.685
            self.h0 = 67.31
            self.s8 = 0.829
        
class Constants:
    def __init__(self):
        self.ERRTOL = 1e-12
        self.G_CGS = 6.67259e-08
        self.MSUN_CGS = 1.98900e+33
        self.MPC2CM = 3.085678e+24
    
class Halo_tools:
    def __init__(self):
        pass
    
    #NFW cumulative mass distribution
    def m_x(self,x):
        ans = np.log(1 + x) - x/(1+x)
        return ans
    #hubble function
    def E_z(self,z):
        ans = np.sqrt(cp.om * (1 + z)**3 + cp.ol)
        return ans
    #critical density as a function of z
    def rhoc(self,z):
        ans = cp.rho_crit0*HP.E_z(z)**2
        return ans
    
    #spherical overdensity radius w.r.t. the critical density
    def rdel_c(self,M,z,delta):
        ans = (3 * M / (4 * np.pi * delta*HP.rhoc(z)))**(1.0/3.0)
        return ans
    
    #spherical overdensity radius w.r.t. the mean matter density
    def rdel_m(self,M,z,delta):
        ans = (3 * M / (4 * np.pi * delta*cp.rho_crit0*cp.om*(1.+z)**3))**(1.0/3.0) 
        return ans

    #Seljak 2000 with hs in units
    def con_M_rel_seljak(self,Mvir, z):
        ans = 5.72 / (1 + z) * (Mvir / 10**14)**(-0.2)
        return ans
    
    #Duffy 2008 with hs in units
    def con_M_rel_duffy(self,Mvir, z):
        ans = 5.09 / (1 + z)**0.71 * (Mvir / 10**14)**(-0.081)
        return ans
        
    #Duffy 2008 with hs in units MEAN DENSITY 200
    def con_M_rel_duffy200(self,Mvir, z):
        ans = 10.14 / (1 + z)**(1.01) * (Mvir / 2e12)**(-0.081)
        return ans
   
    #Mass conversion critical to mean overdensity, needed because the Tinker Mass function uses mean matter
    def Mass_con_del_2_del_mean200(self,Mdel,delta,z):
        Mass = 2.*Mdel
        rdels = HP.rdel_c(Mdel,z,delta)
        ans = Mass*0.0
        for i in xrange(np.size(Mdel)):
            while np.abs(ans[i]/Mass[i] - 1) > c.ERRTOL : 
                ans[i] = Mass[i]
                conz = HP.con_M_rel_duffy200(Mass[i],z) #DUFFY
                rs = HP.rdel_m(Mass[i],z,200)/conz
                xx = rdels[i] / rs
                Mass[i] = Mdel[i] * HP.m_x(conz) / HP.m_x(xx)
        ## Finish when they Converge
        return ans
    
class Halo_tools:
    def __init__(self):
        pass
    

Populating the interactive namespace from numpy and matplotlib


In [8]:
c = Constants()
cp = CosmoParams('Planck15')
HP = Halo_tools()

print HP.m_x(1.0)

h0 = 100.

rho_crit0 = 3. / (8 * np.pi) * (h0 * 1e5)**2 / c.G_CGS * c.MPC2CM / c.MSUN_CGS
print rho_crit0

print HP.Mass_con_del_2_del_mean200(np.array([1e14,2e14]),200,0.1)

0.19314718056
2.77525424593e+11
[  1.34953227e+14   2.72381792e+14]
